<table style="background-color:#F5F5F5;" width="100%">
<tr><td style="background-color:#F5F5F5;">    <tr><td>
            <h2><center>Aprendizagem Automática em Engenharia Biomédica</center></h2>
            <h3><center>1st Semester - 2024/2025</center></h3>
            <h4><center>Universidade Nova de Lisboa - Faculdade de Ciências e Tecnologia</center></h4>
</td></tr>
    <tr><td><h2><b><center>Project</center></b></h2>
    <h4><i><b><center>Predicting Cervical Cancer: A Machine Learning Approach Using Risk Factor Analysis 
</center></b></i></h4></td></tr>
</table>


 <h3>Requesitos nos  modelos </h3>
 <li> Comparison of at least 3 models</li>
 <li>Cross-Validation</li>
 <li> Grid Search </li>
 <li> Results evaluation and discussion </li>


<h3>Avaliação </h3>

 <li> Quality of coding (temos de tipo comentar e explicar tudo)</li>
 <li>Creativity</li>
 <li> Comparison with the state of the Art </li>



<h3>1. Introduction</h3>

Cervical cancer is the fourth most common cancer worldwide and a significant cause of mortality, particularly in low- and middle-income countries, where 94% of all deaths occur [1]. Although medicine has evolved, implementing cytology-based and other types of screening remains challenging in these countries due to the lack of healthcare infrastructure and trained professionals [2]. 
With that in mind, machine learning can be used as a helpful system for interpreting complex datasets and support clinical decision-making due to its strong data analysis capabilities [3]. 

In clinical cases, the most common ways to detect this type of cancer include **the Cytology method, the biopsy, the Hinselmann test, and the Schiller test**. **The Cytology method** involves a microscopic analysis of tissue collected from the cervix to identify cell growth and its intensity. **The biopsy** is a surgical diagnostic procedure that involves collecting living tissue samples to determine if the sample is malignant or benign. **The Hinselmann test** is a visual inspection of the cervix using an iodine solution. However, the results of this test can vary depending on the examiner's expertise. **The Schiller test**, on the other hand, involves administering Lugol’s iodine for visual inspection of suspicious areas on the cervix. To reduce the high mortality rate of cervical cancer, it is crucial for women over 30 years of age to undergo regular screening tests every 3–5 years. In underdeveloped countries, however, frequent screening has been challenging due to limited medical infrastructure and expertise. Thus, we saw an opportunity for a biomedical solution that could renovate this practice and make it more accesible.


Altough the initial approach was to use machine learning characteristics' combined with the risk factors, in order to predict the presence of the cancer, upon analyzing the data and being more familiarized with the problem itself , it occured to us we could rewire the project to predict the results of the main four tests used to detect cervical cancer (Hinselmann, Schille, Citology and Biopsy). This work could be relevant for the management of clinical resources. For example, a person exhibiting certain risk factors could be called in to undergo only two of the four exams, saving both financial and material resource. 

Nevertheless, in order to fulfill the initial approach, we decided to use a model to predict 

 <h3>2. Data Preparation </h3>

The data preparation phase is a critical step in the development of any machine learning model. This phase involves transforming raw data into a clean, organized, and usable format, ensuring it is suitable for analysis and modeling. The primary goal of data preparation is to address inconsistencies, fill in missing values, and convert data into a format that aligns with the requirements of machine learning algorithms.

<h4> 2.1 Data import </h4>

In [331]:
#!pip install ucimlrepo
import pandas as pd  
import numpy as np 
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo 
from typing import Tuple
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [332]:
# fetch dataset 
cervical_cancer_risk_factors = fetch_ucirepo(id=383) 
  
# data (as pandas dataframes) 
df_cervical_cancer = cervical_cancer_risk_factors.data.features 

#Getting dataset dimensions
n_rows = df_cervical_cancer.shape[0]
n_features = df_cervical_cancer.shape[1]
print('The dataset has {} samples and {} features. \n'.format(n_rows, n_features))

#Showing the first 10 rows of the dataset
print('The first 10 rows are displayed below. \n\n')
df_cervical_cancer.head(10)


The dataset has 858 samples and 36 features. 

The first 10 rows are displayed below. 




,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
2,34,1.0,NaN,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.000000,37.0,1.0,3.0,0.0,...,NaN,NaN,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.000000,0.0,1.0,15.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
5,42,3.0,23.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0,0,0
6,51,3.0,17.0,6.0,1.0,34.000000,3.4,0.0,0.0,1.0,...,NaN,NaN,0,0,0,0,1,1,0,1
7,26,1.0,26.0,3.0,0.0,0.000000,0.0,1.0,2.0,1.0,...,NaN,NaN,0,0,0,0,0,0,0,0
8,45,1.0,20.0,5.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,1,0,1,1,0,0,0,0
9,44,3.0,15.0,NaN,1.0,1.266973,2.8,0.0,0.0,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0


<h5> 2.1.1 Data types </h5>
<p style="font-size:15px;"> The data is divided in integer values such as age and number of diagnosis, floats, like the age at which the patient had its first sexual intercourse and a binary classification (0 as false and 1 as true) to evaluate the results of the tests, presence of STD's etc. </p>

In [334]:
print(df_cervical_cancer.dtypes)

Age                                     int64
Number of sexual partners             float64
First sexual intercourse              float64
Num of pregnancies                    float64
Smokes                                float64
Smokes (years)                        float64
Smokes (packs/year)                   float64
Hormonal Contraceptives               float64
Hormonal Contraceptives (years)       float64
IUD                                   float64
IUD (years)                           float64
STDs                                  float64
STDs (number)                         float64
STDs:condylomatosis                   float64
STDs:cervical condylomatosis          float64
STDs:vaginal condylomatosis           float64
STDs:vulvo-perineal condylomatosis    float64
STDs:syphilis                         float64
STDs:pelvic inflammatory disease      float64
STDs:genital herpes                   float64
STDs:molluscum contagiosum            float64
STDs:AIDS                         

In [335]:
#Get all the columns with a binary classification 
binary_columns = df_cervical_cancer.loc[:, (df_cervical_cancer.isin([0, 1]) | df_cervical_cancer.isna()).all()]
binary_columns.describe().iloc[[0]]

,Smokes,Hormonal Contraceptives,IUD,STDs,STDs:condylomatosis,STDs:cervical condylomatosis,STDs:vaginal condylomatosis,STDs:vulvo-perineal condylomatosis,STDs:syphilis,STDs:pelvic inflammatory disease,...,STDs:Hepatitis B,STDs:HPV,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
count,845.0,750.0,741.0,753.0,753.0,753.0,753.0,753.0,753.0,753.0,...,753.0,753.0,858.0,858.0,858.0,858.0,858.0,858.0,858.0,858.0


In [336]:
#Get the columns with continuos values
continuous_columns = df_cervical_cancer.drop(binary_columns.columns, axis=1)
continuous_columns.describe().iloc[[0,1,2,3,7]]

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes (years),Smokes (packs/year),Hormonal Contraceptives (years),IUD (years),STDs (number),STDs: Number of diagnosis,STDs: Time since first diagnosis,STDs: Time since last diagnosis
count,858.000000,832.000000,851.000000,802.000000,845.000000,845.000000,750.000000,741.000000,753.000000,858.000000,71.000000,71.000000
mean,26.820513,2.527644,16.995300,2.275561,1.219721,0.453144,2.256419,0.514804,0.176627,0.087413,6.140845,5.816901
std,8.497948,1.667760,2.803355,1.447414,4.089017,2.226610,3.764254,1.943089,0.561993,0.302545,5.895024,5.755271
min,13.000000,1.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,84.000000,28.000000,32.000000,11.000000,37.000000,37.000000,30.000000,19.000000,4.000000,3.000000,22.000000,22.000000


In [337]:
df_cervical_cancer.isnull().sum()

Age                                     0
Number of sexual partners              26
First sexual intercourse                7
Num of pregnancies                     56
Smokes                                 13
Smokes (years)                         13
Smokes (packs/year)                    13
Hormonal Contraceptives               108
Hormonal Contraceptives (years)       108
IUD                                   117
IUD (years)                           117
STDs                                  105
STDs (number)                         105
STDs:condylomatosis                   105
STDs:cervical condylomatosis          105
STDs:vaginal condylomatosis           105
STDs:vulvo-perineal condylomatosis    105
STDs:syphilis                         105
STDs:pelvic inflammatory disease      105
STDs:genital herpes                   105
STDs:molluscum contagiosum            105
STDs:AIDS                             105
STDs:HIV                              105
STDs:Hepatitis B                  

<p style="font-size:15px;"> As colunas  'STDs: Time since first diagnosis' e 'STDs: Time since last diagnosis' have a lot of missing values  então vamos droppar (n me apetece escrever em ingles)</p>

In [339]:
df_cervical_cancer= df_cervical_cancer.drop(['STDs: Time since first diagnosis','STDs: Time since last diagnosis'], axis=1)
df_cervical_cancer.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
2,34,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0


In [340]:
df_cervical_cancer = df_cervical_cancer.dropna()
print(df_cervical_cancer.shape[0])

668


In [341]:
zero_per_columns = (df_cervical_cancer == 0).sum()
print(zero_per_columns)

Age                                     0
Number of sexual partners               0
First sexual intercourse                0
Num of pregnancies                     14
Smokes                                572
Smokes (years)                        572
Smokes (packs/year)                   572
Hormonal Contraceptives               238
Hormonal Contraceptives (years)       238
IUD                                   593
IUD (years)                           593
STDs                                  603
STDs (number)                         603
STDs:condylomatosis                   631
STDs:cervical condylomatosis          668
STDs:vaginal condylomatosis           664
STDs:vulvo-perineal condylomatosis    632
STDs:syphilis                         653
STDs:pelvic inflammatory disease      667
STDs:genital herpes                   667
STDs:molluscum contagiosum            667
STDs:AIDS                             668
STDs:HIV                              655
STDs:Hepatitis B                  

<p style="font-size:15px;">STDs:AIDS e STDs:cervical condylomatosis  têm todos os valores = 0 pelo q vamos dropar  </p>

In [343]:
df_cervical_cancer= df_cervical_cancer.drop(['STDs:AIDS','STDs:cervical condylomatosis'], axis=1)
df_cervical_cancer.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
5,42,3.0,23.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0


In [344]:
list_nonbinary = ['Number of sexual partners', 'First sexual intercourse' ,'Num of pregnancies', 'Smokes (years)', 'Smokes (packs/year)', 'Hormonal Contraceptives (years)','IUD (years)'] 
list_binary = ['Smokes','Hormonal Contraceptives', 'IUD']

for col in list_nonbinary:
    mean = df_cervical_cancer[col].mean()
    df_cervical_cancer[col] = df_cervical_cancer[col].fillna(mean)
    
for col in list_binary:
    mean = df_cervical_cancer[col].mean()
    df_cervical_cancer[col] = df_cervical_cancer[col].fillna(round(mean))
    
df_cervical_cancer.isnull().sum()

Age                                   0
Number of sexual partners             0
First sexual intercourse              0
Num of pregnancies                    0
Smokes                                0
Smokes (years)                        0
Smokes (packs/year)                   0
Hormonal Contraceptives               0
Hormonal Contraceptives (years)       0
IUD                                   0
IUD (years)                           0
STDs                                  0
STDs (number)                         0
STDs:condylomatosis                   0
STDs:vaginal condylomatosis           0
STDs:vulvo-perineal condylomatosis    0
STDs:syphilis                         0
STDs:pelvic inflammatory disease      0
STDs:genital herpes                   0
STDs:molluscum contagiosum            0
STDs:HIV                              0
STDs:Hepatitis B                      0
STDs:HPV                              0
STDs: Number of diagnosis             0
Dx:Cancer                             0


In [345]:
#Correlação entre as target variables
import seaborn as sns
import matplotlib.pyplot as plt
df_corr = df_cervical_cancer.drop(['Dx'],axis=1).corr()

threshold = 0.5
df_corr_filtered = df_corr[df_corr > threshold]

pairs = []
for i in range(len(df_corr_filtered.columns)):
    for j in range(i):
        if df_corr_filtered.iloc[i, j] > threshold and i!=j:
            pairs.append((df_corr_filtered.columns[i], df_corr_filtered.columns[j], df_corr_filtered.iloc[i, j]))

# Show the pairs and their correlation values
for pair in pairs:
    print(f"Features: {pair[0]} and {pair[1]} have a correlation of {pair[2]:.2f}")


Features: Num of pregnancies and Age have a correlation of 0.56
Features: Smokes (years) and Smokes have a correlation of 0.72
Features: Smokes (packs/year) and Smokes (years) have a correlation of 0.72
Features: IUD (years) and IUD have a correlation of 0.75
Features: STDs (number) and STDs have a correlation of 0.92
Features: STDs:condylomatosis and STDs have a correlation of 0.74
Features: STDs:condylomatosis and STDs (number) have a correlation of 0.90
Features: STDs:vulvo-perineal condylomatosis and STDs have a correlation of 0.73
Features: STDs:vulvo-perineal condylomatosis and STDs (number) have a correlation of 0.89
Features: STDs:vulvo-perineal condylomatosis and STDs:condylomatosis have a correlation of 0.99
Features: STDs: Number of diagnosis and STDs have a correlation of 0.91
Features: STDs: Number of diagnosis and STDs (number) have a correlation of 0.90
Features: STDs: Number of diagnosis and STDs:condylomatosis have a correlation of 0.71
Features: STDs: Number of diagno

In [431]:
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

min_max_scaler = MinMaxScaler(feature_range=(0, 1))

# fit the scaler to train set and scale the train data (this will return a numpy.array)
X_train_minmax = min_max_scaler.fit_transform(X_train)

# scale the test data using the scaler that was fitted on the training data
X_test_minmax = min_max_scaler.transform(X_test)

# converting the training and test data back to a pandas.DataFrame (using the column names of the X DataFrame)
X_train_minmax = pd.DataFrame(X_train_minmax, columns=X.columns)
X_test_minmax = pd.DataFrame(X_test_minmax, columns=X.columns)

 <h3>3. Model Training </h3>

In [348]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, RocCurveDisplay
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from tqdm import tqdm 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import make_scorer, f1_score


<p style="font-size:15px;">FUNÇÃO BLABLABLA EXPLICA </p>

In [435]:
def calculate_metrics(model_name: str, model_instance, X_train, y_train, X_test, y_test) -> dict:
    """
    Calculate performance metrics for a model based on its type.

    :param model_name: Name of the model.
    :param model_instance: The model instance.
    :param X_train: Training features.
    :param y_train: Training labels.
    :param X_test: Test features.
    :param y_test: Test labels.
    :return: Dictionary of metrics.
    """    
    # Predictions
    y_pred_train = model_instance.predict(X_train)
    y_pred_test = model_instance.predict(X_test)

    metrics = {"Train RMSE":np.sqrt(mean_squared_error(y_train, y_pred_train)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_pred_test)),
    "Test R²":r2_score(y_test, y_pred_test)
    }
    
    return metrics

<p style="font-size:15px;">EXPLICA BLABLA  </p>

In [437]:
from typing import Tuple, Union, Dict, List
import numpy as np
import pandas as pd
from tqdm import tqdm


def train_and_evaluate_multi_target(
    model: Tuple[str, any],
    X_train: Union[np.ndarray, pd.DataFrame],
    Y_train: Union[np.ndarray, pd.DataFrame],
    X_test: Union[np.ndarray, pd.DataFrame],
    Y_test: Union[np.ndarray, pd.DataFrame],
) -> pd.DataFrame:
    """
    Train and evaluate a machine learning model for multiple target variables.

    :param model: A tuple containing the model name as a string and the model instance.
    :param X_train: Training features (DataFrame or numpy array).
    :param Y_train: Training labels (DataFrame or numpy array with multiple columns).
    :param X_test: Test features (DataFrame or numpy array).
    :param Y_test: Test labels (DataFrame or numpy array with multiple columns).
    :return: A DataFrame containing performance metrics for each target variable.
    """
    model_name, model_instance = model
    metrics_list = []

    # Iterate over each target variable
    for i in tqdm(range(Y_train.shape[1]), desc="Processing Targets"):
        y_train = Y_train.iloc[:, i]
        y_test = Y_test.iloc[:, i]
        target_name = Y_train.columns[i]
        
        # Train the model
        model_instance.fit(X_train, y_train)
        metrics = calculate_metrics(model_name, model_instance, X_train, y_train, X_test, y_test)
        metrics["Target"] = target_name  # Add target-specific information
        metrics_list.append(metrics)

    # Create DataFrame for metrics
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df = metrics_df.set_index("Target")
    
    return metrics_df


<p style="font-size:15px;"> EXPLICA </p>

In [391]:
#Preparar os Dados - meti aqui por ser uma coisa geral a todos os modelos
Y = df_cervical_cancer.loc[:,['Dx']]
X = df_cervical_cancer.drop(['Dx'],axis=1)

<h4>3.5 Random Forest</h4>

In [463]:
from sklearn.ensemble import RandomForestRegressor

random_forest = ('Random Forest', RandomForestRegressor(random_state=42))

# Treinar e avaliar
model_metrics_rf = train_and_evaluate_multi_target(random_forest, X_train_minmax, y_train, X_test_minmax, y_test)

# Mostrar o DataFrame com formatação tabular
model_metrics_rf

Processing Targets: 100%|██████████| 1/1 [00:00<00:00, 10.03it/s]


,Train RMSE,Test RMSE,Test R²
Target,,,
Dx,0.023151,0.092212,0.421668


<h4>3.2 - Polynomial Regression
</h4>

In [441]:
from sklearn.preprocessing import PolynomialFeatures

# Cria um objeto de features polinomiais de grau 3 -> veio de try and error
poly = PolynomialFeatures(degree=3)

# Ajusta o objeto aos dados de treinamento
poly = poly.fit(X_train)

# Transforma os dados de treinamento e teste. O código também converte a saída em um pandas.DataFrame
X_train_poly = pd.DataFrame(poly.transform(X_train), columns=poly.get_feature_names_out())
X_test_poly = pd.DataFrame(poly.transform(X_test), columns=poly.get_feature_names_out())

# Inicializa o modelo de regressão linear
lin_reg = ('Polynomial Regression', LinearRegression())

# Treinar e avaliar
model_metrics_poly = train_and_evaluate_multi_target(lin_reg, X_train_poly , y_train, X_test_poly, y_test)

# Mostrar o DataFrame com formatação tabular
model_metrics_poly


Processing Targets: 100%|██████████| 1/1 [00:00<00:00,  4.37it/s]


,Train RMSE,Test RMSE,Test R²
Target,,,
Dx,2.971166e-14,0.081973,0.542971


<h4>3.3 Support Vector Machine </h4>

In [457]:
from sklearn.svm import SVR

SVR = ('SVR', SVR())

# Treinar e avaliar
model_metrics_svr = train_and_evaluate_multi_target(SVR, X_train_minmax, y_train, X_test_minmax, y_test)

# Mostrar o DataFrame com formatação tabular
model_metrics_svr

Processing Targets: 100%|██████████| 1/1 [00:00<00:00, 62.67it/s]


,Train RMSE,Test RMSE,Test R²
Target,,,
Dx,0.087929,0.115442,0.093578


In [467]:
from sklearn.model_selection import GridSearchCV

# Definir o grid de parâmetros
param_grid = {
    "C": [0.1, 1, 10, 100],
    "epsilon": [0.01, 0.1, 1],
    "kernel": ["rbf", "linear", "poly"],
    "gamma": ["scale", "auto"]
}

# Otimização
grid_search = GridSearchCV(SVR, param_grid, cv=5, scoring="r2", verbose=1)
grid_search.fit(X_train, y_train)

# Melhor modelo
best_svr = grid_search.best_estimator_
print("Melhores parâmetros:", grid_search.best_params_)

InvalidParameterError: The 'estimator' parameter of GridSearchCV must be an object implementing 'fit'. Got ('SVR', SVR()) instead.

 <h3>4. Result evaluation</h3>

 <h3>5. Discussion</h3>

Principais Barreiras e obstaculos
Perspetivas de melhoria

 <h3>6. Model test (onde fazemos o questionário para mostrar a aplicação do nosso modelo</h3>